# Apache Spark 基本練習：

是一個開源叢集運算框架，最初是由加州大學柏克萊分校AMPLab所開發。相對於Hadoop的MapReduce會在執行完工作後將中介資料存放到磁碟中，Spark使用了記憶體內運算技術，能在資料尚未寫入硬碟時即在記憶體內分析運算。Spark在記憶體內執行程式的運算速度能做到比Hadoop MapReduce的運算速度快上100倍。

Some References :

1. [http://www.mccarroll.net/blog/pyspark/index.html]
2. [https://www.codementor.io/spark/tutorial/spark-python-rdd-basics]
3. [http://backtobazics.com/big-data/spark/apache-spark-map-example/]
4. [http://datascience-enthusiast.com/Python/Apache_Spark1.html]

## RDD 基本操作練習：



### 產生一個整數隨機陣列： python語法：

In [2]:
import numpy as np
random_array = np.random.randint(1000, size=1000)
print (random_array)

# 將資料轉成RDD，分別擺放於各spark executors上

<img src="https://www.dropbox.com/s/br94ete5q3rj9w3/spark%20data%20model.png?dl=1" width="500" align="left">




___
<img src="https://www.dropbox.com/s/l2gohpmn53jkv1b/%20spark%20system%20overview.png?dl=1" width="500" align="left">

In [4]:
rdd = sc.parallelize(random_array)

In [5]:
print type(random_array)
print type(rdd)
print type(rdd.collect())

### RDD為Apache Spark最核心之概念，有別於MapRduce，僅提供Map()與Reduce()兩項操作。 RDD提供兩大類別Transformation與Action

<img src="https://camo.githubusercontent.com/caefb907df659b5cdd384428e575a29f6697f9f4/687474703a2f2f666569736b792e78797a2f70616765732f696d616765732f737061726b2d7472616e73666f726d6174696f6e2d6c6973742e706e67" width="800" align="left">

> <img src="https://www.dropbox.com/s/omfoi3uzcgapcm4/rdd%20transformation%20concept.png?dl=1" width="500" align="left">

> <img src="https://www.dropbox.com/s/3u8gt5376qq5vjy/spark%20core.png?dl=1" width="500" align="left">

## 最基本之Action操作  
### 使用 collect( ) 將分散於各機器之資料，收集成為單機資料集
<img src="https://www.dropbox.com/s/pjv20pl5wkevjf6/collect.png?dl=1" width="500" align="left">

In [10]:
rdd.collect()

## Transformation 觀念介紹

In [12]:
def doubling(x):
    return x*2

print doubling(10)

In [13]:
rdd.take(2)

In [14]:
rdd.map(doubling).take(2)

#### 注意：匿名函式的使用  
    rdd.map(doubling) = rdd.map(lambda x: x*2)

In [16]:
rdd.map(lambda x: x*2).take(2)

#### 練習：使用map()，將所有數字開平方根 
    import math
    print math.sqrt(5)

In [18]:
import math
rdd.map(lambda x: math.sqrt(x)).take(2)

#### 匿名函式的差異

In [20]:
import math
def sqrt(x):
    return math.sqrt(x)

In [21]:
rdd.take(5)

In [22]:
def checkeven(x):
  if x%2==0:
    return x

In [23]:
rdd.map(checkeven).take(5)

In [24]:
rdd.map(sqrt).take(2)

#### 使用filter()，將所有偶數留下，奇數刪除。

In [26]:
rdd.filter(lambda x: x%2==0).take(10)

#### 使用sample( ) 抽樣給定比例之RDD子集合

In [28]:
subsetrdd = rdd.sample(False, 0.01)
subsetrdd.collect()

#### 使用 groupBy( ) 將資料分組

In [30]:
result = subsetrdd.groupBy(lambda x: x%2==0).collect()
print result
print [(x, list(y)) for (x, y) in result]

#### 使用map ( ) 產生 key value pair

In [32]:
keyValueRdd = rdd.map(lambda x: ('even', x) if x%2==0 else ('odd',x))

#### 使用 groupbyKey ( ) 根據key將資料分組

In [34]:
for x in keyValueRdd.groupByKey().collect():
    print x[0], list(x[1])

#### 使用 reduceByKey( )   將具有相同key值之(key, vlaue) pair進行運算

In [36]:
keyValueRdd.reduceByKey(lambda x,y: x+y).take(2)

#### 使用mapValues( ) Key-Value pair之 value值進行 map transformation 
例如：如何計算奇數偶數數目

In [38]:
keyValueRdd.groupByKey().mapValues(lambda x: len(x)).collect()

___
___
___


# RDD Action Action Operation
#### 使用reduce()，計算所有數之加總值。

In [40]:
rdd.reduce(lambda x,y:x*x+y*y)

In [41]:
rdd.reduce(lambda a,b:a+b)

______
______
______

## 觀念
### 練習：使用map(), reduce()，計算所有數字平方和。

In [44]:
rdd.map(lambda x: x*x).reduce(lambda x,y:x+y)

## 練習：使用map(), reduceByKey()，計算所有數字出現頻率。

In [46]:
rdd.map(lambda x: (x,1)).reduceByKey(lambda a,b:a+b).collect()

_____
_____
_____

# WordCount

In [48]:
!ls

In [49]:
text_file = sc.textFile("/FileStore/tables/Speech2.txt")

In [50]:
text_file.count()

In [51]:
text_file = sc.textFile("/FileStore/tables/Ep.txt")

In [52]:
text_file.count()

In [53]:
print text_file.take(1)

In [54]:
wordcountsRDD = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
result = wordcountsRDD.take(5)

for i in result:
  print i[0], i[1]

#### 根據字元符號順序做排序

In [56]:
text_file = sc.textFile("/FileStore/tables/Speech2.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b).sortByKey()
counts.take(5)

In [57]:
display(dbutils.fs.ls("/FileStore/tables"))

#### 出現頻率做排序

In [59]:
text_file = sc.textFile("/FileStore/tables/Speech.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False)
sortcount = counts.take(5)

for i in sortcount:
  print i[0], i[1]

#### map 與 flatMap的差異

In [61]:
text_file = sc.textFile("/FileStore/tables/Speech.txt")
counts = text_file.map(lambda line: line.split(" "))
counts.take(5)

In [62]:
text_file = sc.textFile(data_file)
counts = text_file.flatMap(lambda line: line.split(" "))
counts.take(10)

____
____
____
# pi-estimation

In [64]:
import random

def sample(p):
    x, y = random.random(), random.random()
    return 1 if x*x + y*y < 1 else 0

count = sc.parallelize(xrange(0, 100000000)).map(sample) \
             .reduce(lambda a, b: a + b)
print "Pi is roughly %f" % (4.0 * count / 100000000)

In [65]:
count = sc.parallelize(xrange(0, 1000000)).map(lambda p: 1 if (random.random()**2 + random.random()**2)<1 else 0) \
             .reduce(lambda a, b: a + b)
print "Pi is roughly %f" % (4.0 * count / 1000000)

___
___
___

# Filter Example

In [67]:
text_file = sc.textFile("/FileStore/tables/EngOut.txt")
print (text_file.count())

# 練習 Wiki Article Word Count 根據出現頻率做排序

In [69]:
import urllib
urllib.urlretrieve("http://web.nchu.edu.tw/~yfan/EngOut.txt","engwiki")
text_file = sc.textFile("./engwiki")
text_file.take(10000)